In [2]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
 
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer

In [5]:
LANGUAGE = "english"
SENTENCES_COUNT = 10
 
 
    
#url="https://en.wikipedia.org/wiki/Deep_learning"
#parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))

# or for plain text files
parser = PlaintextParser.from_file("../src/Email.txt", Tokenizer(LANGUAGE))



print ("--LsaSummarizer--")    
summarizer = LsaSummarizer()
summarizer = LsaSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = get_stop_words(LANGUAGE)
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

print ("--LuhnSummarizer--")     
summarizer = LuhnSummarizer() 
summarizer = LsaSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = ("I", "am", "the", "you", "are", "me", "is", "than", "that", "this",)
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

print ("--EdmundsonSummarizer--")     
summarizer = EdmundsonSummarizer() 
words = ("deep", "learning", "neural" )
summarizer.bonus_words = words

words = ("another", "and", "some", "next",)
summarizer.stigma_words = words


words = ("another", "and", "some", "next",)
summarizer.null_words = words
for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence) 

--LsaSummarizer--
Hi Jane,
Thank you for keeping me updated on this issue.
I'm happy to hear that the issue got resolved after all and you can now use the app in its full functionality again.
Also many thanks for your suggestions.
We hope to improve this feature in the future.
In case you experience any further problems with the app, please don't hesitate to contact me again.
Best regards,
John Doe Customer Support
1600 Amphitheatre Parkway Mountain View, CA United States
--LuhnSummarizer--
Hi Jane,
Thank you for keeping me updated on this issue.
I'm happy to hear that the issue got resolved after all and you can now use the app in its full functionality again.
Also many thanks for your suggestions.
We hope to improve this feature in the future.
In case you experience any further problems with the app, please don't hesitate to contact me again.
Best regards,
John Doe Customer Support
1600 Amphitheatre Parkway Mountain View, CA United States
--EdmundsonSummarizer--
Hi Jane,
Thank you fo

# By Model

In [1]:
import skipthoughts



ModuleNotFoundError: No module named 'skipthoughts'

In [ ]:
# You would need to download pre-trained models first
model = skipthoughts.load_model()

encoder = skipthoughts.Encoder(model)
encoded = encoder.encode(sentences)